In [3]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 5.3 MB/s 
     |████████████████████████████████| 189 kB 40.0 MB/s 
     |████████████████████████████████| 46 kB 3.3 MB/s 
     |████████████████████████████████| 1.2 MB 35.9 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 51 kB 263 kB/s 
Mounted at /content/gdrive


In [4]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [5]:
path = untar_data(URLs.MNIST_SAMPLE)

In [6]:
URLs.MNIST_SAMPLE

'https://s3.amazonaws.com/fast-ai-sample/mnist_sample.tgz'

In [7]:
Path.BASE_PATH = path
path.ls()

(#3) [Path('train'),Path('valid'),Path('labels.csv')]

In [8]:
(path/'train').ls()

(#2) [Path('train/7'),Path('train/3')]

In [9]:
def get_img_loader(url, train_folder, valid_folder, presize, resize, bs):
  path = untar_data(url)
  dblock = DataBlock(
      blocks = (ImageBlock, CategoryBlock),
      splitter = GrandparentSplitter(train_name=train_folder, valid_name=valid_folder),
      get_items = get_image_files,
      get_y = parent_label,
      item_tfms = Resize(presize),
      batch_tfms = aug_transforms(min_scale=0.5, size=resize)
  )
  dls = dblock.dataloaders(path, bs=bs)
  return dls

In [10]:
dls = get_img_loader(URLs.MNIST_SAMPLE, 'train', 'valid', 28, 28, 64)
x_t, y_t = dls.train.one_batch()

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:1051: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:766.)
  ret = func(*args, **kwargs)


In [11]:
x_t.shape, y_t.shape

(torch.Size([64, 3, 28, 28]), torch.Size([64]))

In [12]:
def _single_conv(ch_in, ch_out, stride, ks=3, act=True, gammaZero=False):
    layers = [nn.Conv2d(ch_in, ch_out, ks, stride=stride, padding=ks//2)]
    bn = nn.BatchNorm2d(ch_out)
    if gammaZero:
      torch.nn.init.zeros_(bn)
    layers.append(bn)

    if act:
      layers.append(self.relu)
    
    return nn.Sequential(*layers)

In [13]:
class ResBlock(Module):
  def __init__(self, ch_in, ch_out, stride, ks=3):
    self.conv = self._res_conv(ch_in, ch_out, stride, ks=ks)
    self.relu = nn.ReLU()
    self.pool = self._return if stride==1 else nn.AvgPool2d(stride)
    self.id_conv = self._return if ch_in == ch_out else self._single_conv(ch_in, ch_out, 1, ks=1, act=False)

  
  def _return(self, x):
    return x
  
  # def _single_conv(self, ch_in, ch_out, stride, ks=3, act=True, gammaZero=False):
  #   layers = [nn.Conv2d(ch_in, ch_out, ks, stride=stride, padding=ks//2)]
  #   bn = nn.BatchNorm2d(ch_out)
  #   if gammaZero:
  #     torch.nn.init.zeros_(bn)
  #   layers.append(bn)

  #   if act:
  #     layers.append(self.relu)
    
  #   return nn.Sequential(*layers)

  def _res_conv(self, ch_in, ch_out, stride, ks=3):
    convs = nn.Sequential(
        _single_conv(ch_in, ch_out, stride, ks),
        _single_conv(ch_out, ch_out, 1, act=False, gammaZero=True)
    )

    return convs
  
  def forward(self, x):
    return self.relu(self.conv(x) + self.id_conv(self.pool(x)))

In [ ]:
class ResNet(Module):
  def __init__(self, ):
    self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    # create a simple convolutional resnet stem
    stem = self._stem([3, 32, 32, 64])
    
  
  def _stem(self, sizes):
    layers = [
              *[_single_conv(sizes[i], sizes[i+1], 2 if i == 0 else 1) for i in range(len(sizes) - 1)],
              self.max_pool
    ]
    return layers